In [1]:
%reload_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from IPython.display import display, Markdown
from collections import Counter
import random
from lightgbm import LGBMClassifier

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler, MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score

from golemai.dataset.custom_column_selectors import JensenShannonSelector, ProportionAggSelector
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from imblearn.under_sampling import RandomUnderSampler


In [23]:
EXP_NAME = 'window_step_1__on_query__all'

In [24]:
os.getcwd()

'c:\\Users\\Piotr.Matys\\Python_Projects\\hallu_project\\Research\\notebooks'

In [25]:
context_df = None

for f_ in os.listdir(os.path.join("..", "..", EXP_NAME)):
    
    if f_.startswith('attension'):

        if context_df is None:
            context_df = pd.read_parquet(os.path.join("..", "..", EXP_NAME, f_))
        else:
            context_df = pd.concat((context_df, pd.read_parquet(os.path.join("..", "..", EXP_NAME, f_))))

context_df = context_df.dropna()

In [26]:
context_df['dataset'].value_counts()

dataset
cnndm          29715
xsum           29628
nq             28271
poquad_v2      21804
bioask         16599
hotpotqa_en    10066
hotpotqa_pl     7392
polqa           5817
Name: count, dtype: int64

In [27]:
context_df['label'].value_counts()

label
0    136221
1     13071
Name: count, dtype: int64

In [28]:
context_df.groupby(['dataset', 'label']).size()

dataset      label
bioask       0        15721
             1          878
cnndm        0        28895
             1          820
hotpotqa_en  0         8746
             1         1320
hotpotqa_pl  0         6454
             1          938
nq           0        24897
             1         3374
polqa        0         4640
             1         1177
poquad_v2    0        20420
             1         1384
xsum         0        26448
             1         3180
dtype: int64

In [29]:
context_df.columns = [
    f'layer_{i}_head_{j}' for i in range(42) for j in range(16)
] + [
    'label', 'dataset'
]

In [30]:
grouped_df = context_df.groupby(['dataset', 'label']).agg(['median']).T

In [31]:
dataset_counts = context_df.groupby(['dataset', 'label']).size().to_frame().reset_index().rename(columns={0: 'count'})

In [32]:
TOP_N = 10

In [33]:
previous_features = []
highest_non_hallu, highest_hallu = [], []

for dataset in grouped_df.columns.levels[0]:

    stats_grouped = grouped_df[dataset].reset_index()

    stats_grouped = stats_grouped.rename(columns={'level_0': 'feature', 'level_1': 'statistic'})

    stats_grouped['median_diff'] = stats_grouped[0] - stats_grouped[1]
    stats_grouped['median_perc_diff'] = stats_grouped['median_diff'] / stats_grouped[0] * 100
    
    stats_grouped.drop(columns=['median_diff', 'statistic'], inplace=True)

    stats_grouped['median_proportion'] = stats_grouped[0] / stats_grouped[1]

    n_ok, n_hallu = dataset_counts.loc[dataset_counts['dataset'] == dataset, 'count'].values

    # display(Markdown(f"### **{dataset}**, hallu: {n_hallu}, non-hallu: {n_ok}"))

    highest_non_hallu_context = stats_grouped.sort_values('median_proportion', ascending=False).head(TOP_N)
    highest_hallu_context = stats_grouped.sort_values('median_proportion', ascending=True).head(TOP_N)

    previous_features.extend(highest_non_hallu_context['feature'].values)
    previous_features.extend(highest_hallu_context['feature'].values)

    highest_hallu.append(highest_hallu_context)
    highest_non_hallu.append(highest_non_hallu_context)

In [34]:
feature_counts = Counter(previous_features)
common_features = {feature for feature, count in feature_counts.items() if count > 1}

# Generate a random color for each common feature
def generate_random_color():
    return f'#{random.randint(0, 0xFFFFFF):06x}'

# Assign colors from the palette to common features
color_map = {feature: generate_random_color() for feature in common_features}

def highlight_common_features(s):
    return [f'color: {color_map[v]}' if v in color_map else '' for v in s]

In [35]:

display(Markdown(f"## **Highest non-hallu**"))
for df, dataset in zip(highest_non_hallu, grouped_df.columns.levels[0]):

    n_ok, n_hallu = dataset_counts.loc[dataset_counts['dataset'] == dataset, 'count'].values
    display(Markdown(f"### **{dataset}**, hallu: {n_hallu}, non-hallu: {n_ok}"))
    
    styled_df = df.style.apply(highlight_common_features, subset=['feature'])
    display(styled_df)

display(Markdown(f"## **Highest hallu**"))
for df, dataset in zip(highest_hallu, grouped_df.columns.levels[0]):

    n_ok, n_hallu = dataset_counts.loc[dataset_counts['dataset'] == dataset, 'count'].values
    display(Markdown(f"### **{dataset}**, hallu: {n_hallu}, non-hallu: {n_ok}"))

    styled_df = df.style.apply(highlight_common_features, subset=['feature'])
    display(styled_df)

del highest_non_hallu, highest_hallu

## **Highest non-hallu**

### **bioask**, hallu: 878, non-hallu: 15721

label,feature,0,1,median_perc_diff,median_proportion
489,layer_30_head_9,0.000179,0.000062,65.193462,2.873023
214,layer_13_head_6,0.000010,0.000004,61.890244,2.624000
207,layer_12_head_15,0.000276,0.000117,57.739637,2.366284
503,layer_31_head_7,0.000334,0.000149,55.375000,2.240896
460,layer_28_head_12,0.000468,0.000210,55.135066,2.228912
237,layer_14_head_13,0.000515,0.000236,54.243756,2.185494
507,layer_31_head_11,0.000471,0.000225,52.263531,2.094834
242,layer_15_head_2,0.000985,0.000486,50.653437,2.026484
243,layer_15_head_3,0.000105,0.000053,49.716714,1.988732
532,layer_33_head_4,0.000361,0.000190,47.377968,1.900345


### **cnndm**, hallu: 820, non-hallu: 28895

label,feature,0,1,median_perc_diff,median_proportion
621,layer_38_head_13,0.000108,0.000077,28.977901,1.408012
259,layer_16_head_3,0.000627,0.000467,25.456274,1.341495
247,layer_15_head_7,0.001472,0.001104,25.000000,1.333333
341,layer_21_head_5,0.000808,0.000609,24.631268,1.326810
149,layer_9_head_5,0.000030,0.000022,24.596774,1.326203
195,layer_12_head_3,0.002037,0.001536,24.578652,1.325885
335,layer_20_head_15,0.001163,0.000891,23.318294,1.304092
663,layer_41_head_7,0.000161,0.000124,22.949002,1.297842
328,layer_20_head_8,0.001837,0.001421,22.637591,1.292617
351,layer_21_head_15,0.001934,0.001513,21.745562,1.277883


...

label,feature,0,1,median_perc_diff,median_proportion
163,layer_10_head_3,0.007454,0.014862,-99.385875,0.501540
373,layer_23_head_5,0.004131,0.008102,-96.121884,0.509887
86,layer_5_head_6,0.001729,0.003162,-82.951724,0.546592
149,layer_9_head_5,0.000113,0.000206,-82.172996,0.548929
153,layer_9_head_9,0.000977,0.001684,-72.418843,0.579983
138,layer_8_head_10,0.007795,0.012886,-65.304624,0.604944
340,layer_21_head_4,0.007059,0.011627,-64.712240,0.607119
283,layer_17_head_11,0.004547,0.007206,-58.473154,0.631022
208,layer_13_head_0,0.004999,0.007748,-54.979016,0.645249
258,layer_16_head_2,0.002633,0.004059,-54.147048,0.648731


### **poquad_v2**, hallu: 1384, non-hallu: 20420

label,feature,0,1,median_perc_diff,median_proportion
236,layer_14_head_12,0.001473,0.004982,-238.232438,0.295655
340,layer_21_head_4,0.002726,0.007483,-174.527642,0.364262
373,layer_23_head_5,0.001970,0.004999,-153.727009,0.394124
304,layer_19_head_0,0.004766,0.010792,-126.410564,0.441676
283,layer_17_head_11,0.002211,0.004940,-123.468507,0.447490
345,layer_21_head_9,0.002654,0.005863,-120.912684,0.452668
163,layer_10_head_3,0.004789,0.009838,-105.416169,0.486817
208,layer_13_head_0,0.003451,0.007067,-104.752694,0.488394
408,layer_25_head_8,0.002662,0.005281,-98.423504,0.503973
293,layer_18_head_5,0.004555,0.008831,-93.886097,0.515767


### **xsum**, hallu: 3180, non-hallu: 26448

label,feature,0,1,median_perc_diff,median_proportion
147,layer_9_head_3,0.000004,0.000006,-53.846154,0.650000
236,layer_14_head_12,0.000038,0.000053,-40.253566,0.712994
237,layer_14_head_13,0.000195,0.000267,-37.167329,0.729037
452,layer_28_head_4,0.000285,0.000376,-31.829574,0.758555
212,layer_13_head_4,0.000086,0.000112,-30.000000,0.769231
384,layer_24_head_0,0.000099,0.000124,-25.483092,0.796920
489,layer_30_head_9,0.000072,0.000090,-24.731627,0.801721
122,layer_7_head_10,0.000077,0.000097,-24.653846,0.802222
458,layer_28_head_10,0.000203,0.000253,-24.281525,0.804625
394,layer_24_head_10,0.000194,0.000240,-23.558282,0.809335


### Jensen Shanon features

In [36]:
# context_df = context_df.loc[context_df['dataset'].isin(['cnndm', 'nq'])]

In [37]:
TOP_N_FEATURES = 50
SPLIT_VAL = True

In [38]:
lgbm_config = {
    'n_estimators': 250,
    'learning_rate': 0.001,
    'max_depth': 10,
    'num_leaves': 15,
    'random_state': 42,
    'n_jobs': -1,
    'silent': True,
    'verbose': -1,
}

log_reg_config = {
    'max_iter': 10000,
    'random_state': 42,
    'class_weight': 'balanced',
    'n_jobs': -1
}

In [39]:
from imblearn.pipeline import Pipeline
from imblearn import FunctionSampler

def random_undersample(X, y):
    return RandomUnderSampler(random_state=42).fit_resample(X, y)

models = {
    'lgbm_js': Pipeline(
        [
            ('selector', JensenShannonSelector(n_features=TOP_N_FEATURES)),
            ('sampler', FunctionSampler(func=random_undersample)),
            ('classifier', LGBMClassifier(
                **lgbm_config
            )),
        ]
    ),
    'lgbm_pa': Pipeline(
        [
            ('selector', ProportionAggSelector(n_features=TOP_N_FEATURES)),
            ('sampler', FunctionSampler(func=random_undersample)),
            ('classifier', LGBMClassifier(
                **lgbm_config
            )),
        ]
    ),
    'lgbm_all': Pipeline(
        [
            ('sampler', FunctionSampler(func=random_undersample)),
            ('classifier', LGBMClassifier(
                **lgbm_config
            )),
        ]
    ),
    'logistic_reg_js': Pipeline(
        [
            ('selector', JensenShannonSelector(n_features=TOP_N_FEATURES)),
            ('sampler', FunctionSampler(func=random_undersample)),
            ('scaler', RobustScaler()),
            ('classifier', LogisticRegression(**log_reg_config)),
        ]
    ),
    'logistic_reg_pa': Pipeline(
        [
            ('selector', ProportionAggSelector(n_features=TOP_N_FEATURES)),
            ('sampler', FunctionSampler(func=random_undersample)),
            ('scaler', RobustScaler()),
            ('classifier', LogisticRegression(**log_reg_config)),
        ]
    ),
    'logistic_reg_all': Pipeline(
        [
            ('sampler', FunctionSampler(func=random_undersample)),
            ('scaler', RobustScaler()),
            ('classifier', LogisticRegression(**log_reg_config)),
        ]
    ),
}

In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn.base import clone

validation_results = []

# Loop over each unique dataset
for dataset in context_df['dataset'].unique():

    in_dist_sample = context_df.loc[context_df['dataset'] != dataset]
    out_dist_sample = context_df.loc[context_df['dataset'] == dataset]

    for model_name in models.keys():

        X_train, X_test = in_dist_sample.drop(columns=['label', 'dataset']), out_dist_sample.drop(columns=['label', 'dataset'])
        y_train, y_test = in_dist_sample['label'], out_dist_sample['label']

        skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
        aucs_train, aucs_val, aucs_test = [], [], []

        for train_index, val_index in skf.split(X_train, y_train):
            
            X_tr, X_val = X_train.iloc[train_index], X_train.iloc[val_index]
            y_tr, y_val = y_train.iloc[train_index], y_train.iloc[val_index]

            model = clone(models[model_name])

            model.fit(X_tr, y_tr)

            train_pred = model.predict_proba(X_tr)
            aucs_train.append(roc_auc_score(y_tr, train_pred[:, 1]))

            val_pred = model.predict_proba(X_val)
            aucs_val.append(roc_auc_score(y_val, val_pred[:, 1]))

            test_pred = model.predict_proba(X_test)
            aucs_test.append(roc_auc_score(y_test, test_pred[:, 1]))

        train_auc = np.mean(aucs_train)
        auc_val = np.mean(aucs_val)
        auc_test = np.mean(aucs_test)

        print(f'Dataset: {dataset}, Model: {model_name}, Train AUC: {train_auc}, Val AUC: {auc_val}, Test AUC: {auc_test}')

        # Store the result
        validation_results.append({
            'dataset': dataset,
            'columns_selection': model.named_steps['selector'].__class__.__name__ if 'selector' in model.named_steps else 'all',
            'model': model_name,
            'train_auc': train_auc,
            'val_auc': auc_val,
            'test_auc': auc_test
        })

print('\n\n')

Dataset: nq, Model: lgbm_js, Train AUC: 0.7337780600171252, Val AUC: 0.724425824154792, Test AUC: 0.6410814630968387
Dataset: nq, Model: lgbm_pa, Train AUC: 0.7418522873842385, Val AUC: 0.725429323571775, Test AUC: 0.6536936696081751
Dataset: nq, Model: lgbm_all, Train AUC: 0.7702265660502821, Val AUC: 0.7555577746499684, Test AUC: 0.6311142428441218


KeyboardInterrupt: 

: 

In [ ]:
df = pd.DataFrame(validation_results)

# Find the best columns_selection for each dataset based on val_auc
best_columns_selection = df.loc[df.groupby('dataset')['test_auc'].idxmax()]

# Color the best columns_selection for each dataset
def highlight_best_columns_selection(row):
    if row['columns_selection'] == best_columns_selection.loc[best_columns_selection['dataset'] == row['dataset'], 'columns_selection'].values[0]:
        return ['color: yellow'] * len(row)
    else:
        return [''] * len(row)

styled_df = df.style.apply(highlight_best_columns_selection, axis=1)
display(styled_df)

,dataset,columns_selection,model,train_auc,val_auc,test_auc
0,nq,JensenShannonSelector,lgbm_js,0.839207,0.758654,0.493708
1,nq,ProportionAggSelector,lgbm_pa,0.845455,0.771071,0.497544
2,nq,all,lgbm_all,0.890825,0.796437,0.514778
3,nq,JensenShannonSelector,logistic_reg_js,0.749181,0.721784,0.447013
4,nq,ProportionAggSelector,logistic_reg_pa,0.781309,0.753736,0.429978
5,nq,all,logistic_reg_all,0.961068,0.940930,0.441213
6,cnndm,JensenShannonSelector,lgbm_js,0.790798,0.770530,0.496878
7,cnndm,ProportionAggSelector,lgbm_pa,0.792810,0.768284,0.490242
8,cnndm,all,lgbm_all,0.830455,0.805947,0.522745
9,cnndm,JensenShannonSelector,logistic_reg_js,0.745861,0.740620,0.569627
